In [1]:
rawdata = open("105_student.csv",encoding="utf8").read().splitlines()

dataList = []
for line in rawdata:
    dataList.append(line.split(","))
    
stuList = []
for i in dataList[1:]:
    AcaCode = i[0]
    AcaName = i[1]
    DayNight = i[2]
    Level = i[3]
    M1 = int(i[4])
    F1 = int(i[5])
    M2 = int(i[6])
    F2 = int(i[7])
    M3 = int(i[8])
    F3 = int(i[9])
    M4 = int(i[10])
    F4 = int(i[11])
    M5 = int(i[12])
    F5 = int(i[13])
    M6 = int(i[14])
    F6 = int(i[15])
    M7 = int(i[16])
    F7 = int(i[17])
    MX = int(i[18])
    FX = int(i[19])
    City = i[20]
    Typ2 = i[21]
    stuList.append([AcaCode,AcaName,DayNight,Level,M1,F1,M2,F2,M3,F3,M4,F4,M5,F5,M6,F6,M7,F7,MX,FX,City,Typ2])
    
for i in stuList[0:3]:
    print(i)

['0001', '國立政治大學', 'D 日', 'D 博士', 113, 66, 91, 68, 90, 63, 77, 58, 86, 48, 84, 40, 64, 33, 0, 0, '30 臺北市', '1 一般']
['0001', '國立政治大學', 'D 日', 'M 碩士', 617, 755, 622, 712, 352, 422, 217, 265, 0, 0, 0, 0, 0, 0, 0, 0, '30 臺北市', '1 一般']
['0001', '國立政治大學', 'D 日', 'B 學士', 872, 1347, 1017, 1232, 850, 1367, 846, 1398, 0, 0, 0, 0, 0, 0, 327, 455, '30 臺北市', '1 一般']


In [2]:
import sqlite3

dbname = "stuData.db"
table = "universities"
conn = sqlite3.connect(dbname)
c = conn.cursor()
c.execute('CREATE TABLE {tn} (id INTEGER PRIMARY KEY, AcaCode TEXT, AcaName TEXT, DayNight TEXT, Level TEXT, M1 INTEGER, F1 INTEGER, M2 INTEGER, F2 INTEGER, M3 INTEGER, F3 INTEGER, M4 INTEGER, F4 INTEGER, M5 INTEGER, F5 INTEGER, M6 INTEGER, F6 INTEGER, M7 INTEGER, F7 INTEGER, MX INTEGER,FX INTEGER, City TEXT, Typ2 TEXT)'.format(tn=table))
conn.commit()
conn.close()
print("Database",dbname," Successfully Created!")

Database stuData.db  Successfully Created!


In [3]:
dbname = "stuData.db"
table = "universities"
import sqlite3
conn = sqlite3.connect(dbname)
c = conn.cursor()
count = 0
for AcaCode,AcaName,DayNight,Level,M1,F1,M2,F2,M3,F3,M4,F4,M5,F5,M6,F6,M7,F7,MX,FX,City,Typ2 in stuList:
    conn.execute("INSERT INTO universities (AcaCode,AcaName,DayNight,Level,M1,F1,M2,F2,M3,F3,M4,F4,M5,F5,M6,F6,M7,F7,MX,FX,City,Typ2) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",(AcaCode,AcaName,DayNight,Level,M1,F1,M2,F2,M3,F3,M4,F4,M5,F5,M6,F6,M7,F7,MX,FX,City,Typ2))
    count = count + 1
conn.commit()
conn.close()

In [5]:
def queryDB(sqlite_file, QueryStatement):
    import sqlite3
    sqlite_file = sqlite_file
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute(QueryStatement) 
    dbData = []
    for i in out:
        dbData.append([i])
    conn.close()
    print("取得資料行數:", len(dbData))
    return dbData

QueryStatement = "SELECT * FROM universities WHERE `Level` LIKE '%學士%' GROUP BY AcaName;"
dbData = queryDB("stuData.db", QueryStatement)

for i in dbData[0:3]:
    print(i)

取得資料行數: 71
[(336, '1015', '世新大學', 'P 進', 'B 學士', 28, 32, 30, 42, 21, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '30 臺北市', '1 一般')]
[(732, '1125', '中信金融管理學院', 'P 進', 'B 學士', 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, '21 臺南市', '1 一般')]
[(276, '1004', '中原大學', 'D 日', 'B 學士', 1714, 1517, 1834, 1457, 1791, 1409, 1618, 1345, 68, 80, 0, 0, 0, 0, 441, 185, '03 桃園市', '1 一般')]


In [48]:
PriQueryStatement = "WITH tempTable AS ( SELECT * FROM universities WHERE DayNight LIKE '%日%' AND (`Level` LIKE '%學士%' OR `Level` LIKE '%四技%') ) SELECT * FROM tempTable WHERE AcaName NOT LIKE '%國立%' GROUP BY AcaCode;"
dbDataPrivate = queryDB("stuData.db", PriQueryStatement)
privateAca = []
for i in dbDataPrivate:
    privateAca.append([i[0][2],i[0][5:12],i[0][13:20]])
    
PubQueryStatement = "WITH tempTable AS ( SELECT * FROM universities WHERE DayNight LIKE '%日%' AND (`Level` LIKE '%學士%' OR `Level` LIKE '%四技%') ) SELECT * FROM tempTable WHERE AcaName LIKE '%國立%' GROUP BY AcaCode;"
dbDataPublic = queryDB("stuData.db", PubQueryStatement)    
publicAca = []
for i in dbDataPublic:
    publicAca.append([i[0][2],i[0][5:12],i[0][13:20]])
    
extensionPublic = 0
ontimePublic = 0
for i in publicAca:
    ontimePublic = ontimePublic + sum(i[1])    
    extensionPublic = extensionPublic + sum(i[2])
print("國立大學延畢率:",extensionPublic/(ontimePublic+extensionPublic),"延畢人數:",extensionPublic,"如期畢業人數:",ontimePublic)

extensionPrivate = 0
ontimePrivate = 0
for i in privateAca:
    ontimePrivate = ontimePrivate + sum(i[1])    
    extensionPrivate = extensionPrivate + sum(i[2])
print("私立大學延畢率:",extensionPrivate/(ontimePrivate+extensionPrivate),"延畢人數:",extensionPrivate,"如期畢業人數:",ontimePrivate)

取得資料行數: 97
取得資料行數: 48
國立大學延畢率: 0.03687447134398646 延畢人數: 8065 如期畢業人數: 210650
私立大學延畢率: 0.040673808863562685 延畢人數: 20828 如期畢業人數: 491246


In [44]:
MastersQueryStatement = "SELECT * FROM universities WHERE `Level` LIKE '%碩士%'"
dbDataMasters = queryDB("stuData.db", MastersQueryStatement)

bMasters = 0
gMasters = 0
for i in dbDataMasters:
    bMasters = bMasters + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gMasters = gMasters + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("男碩士:",bMasters)
print("女碩士:",gMasters)

PHDQueryStatement = "SELECT * FROM universities WHERE `Level` LIKE '%博士%'"
dbDataPHD = queryDB("stuData.db", PHDQueryStatement)

bPHD = 0
gPHD = 0
for i in dbDataPHD:
    bPHD = bPHD + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gPHD = gPHD + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("男博士:",bPHD)
print("女博士:",gPHD)

取得資料行數: 264
男碩士: 93583
女碩士: 75955
取得資料行數: 83
男博士: 19464
女博士: 9357


In [47]:
BGQueryStatement = "SELECT * FROM universities WHERE DayNight LIKE '%日%' AND ( `Level` LIKE '%學士%' OR `Level` LIKE '%四技%')"
dbDataBG = queryDB("stuData.db", BGQueryStatement)

M1 = 0
M2 = 0
M3 = 0
M4 = 0
F1 = 0
F2 = 0
F3 = 0
F4 = 0
for i in dbDataBG:
    M1 = M1 + i[0][5]
    M2 = M2 + i[0][7]
    M3 = M3 + i[0][9]
    M4 = M4 + i[0][11]
    F1 = F1 + i[0][6]
    F2 = F2 + i[0][8]
    F3 = F3 + i[0][10]
    F4 = F4 + i[0][12]
print("男 大一:",M1,"大二:",M2,"大三:",M3,"大四:",M4)
print("女 大一:",F1,"大二:",F2,"大三:",F3,"大四:",F4)

取得資料行數: 145
男 大一: 98935 大二: 102466 大三: 99975 大四: 96759
女 大一: 99530 大二: 102445 大三: 101786 大四: 99048


In [52]:
NationalQueryStatement = "WITH tempTable AS ( SELECT * FROM universities WHERE `Level` LIKE '%博士%' OR `Level` LIKE '%碩士%' ) SELECT * FROM tempTable WHERE AcaName LIKE '%國立%'"
dbDataNational = queryDB("stuData.db", NationalQueryStatement)

bNational = 0
gNational = 0
for i in dbDataNational:
    bNational = bNational + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gNational = gNational + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("國立男碩博:",bNational)
print("國立女碩博:",gNational)

PrivateQueryStatement = "WITH tempTable AS ( SELECT * FROM universities WHERE `Level` LIKE '%博士%' OR `Level` LIKE '%碩士%' ) SELECT * FROM tempTable WHERE AcaName NOT LIKE '%國立%'"
dbDataPrivate = queryDB("stuData.db", PrivateQueryStatement)

bPrivate = 0
gPrivate = 0
for i in dbDataPrivate:
    bPrivate = bPrivate + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gPrivate = gPrivate + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("私立男碩博:",bPrivate)
print("私立女碩博:",gPrivate)

取得資料行數: 136
國立男碩博: 79309
國立女碩博: 56262
取得資料行數: 211
私立男碩博: 33738
私立女碩博: 29050


In [64]:
GetCities = "SELECT city FROM universities GROUP BY city"
listCities = queryDB("stuData.db", GetCities)

cities = []
for i in listCities:
    cities.append(i[0][0])
print(cities[0][-3:])

取得資料行數: 24
新北市


In [72]:
city = cities[0][-3:]
for city in cities:
    city = city[-3:]
    uniCity = "SELECT * FROM universities WHERE City LIKE '%{}%' AND ( `Level` LIKE '%學士%' OR `Level` LIKE '%四技%')".format(city)
    listUniCity = queryDB("stuData.db", uniCity)

    studentSum = 0
    for i in listUniCity[0:3]:
        studentSum = studentSum + sum(i[0][5:21])
    print("{}:".format(city),studentSum)

取得資料行數: 35
新北市: 10204
取得資料行數: 4
宜蘭縣: 7947
取得資料行數: 16
桃園市: 21207
取得資料行數: 4
新竹縣: 13077
取得資料行數: 5
苗栗縣: 10583
取得資料行數: 32
臺中市: 12503
取得資料行數: 9
彰化縣: 15268
取得資料行數: 3
南投縣: 9105
取得資料行數: 5
雲林縣: 14997
取得資料行數: 7
嘉義縣: 11217
取得資料行數: 24
臺南市: 16108
取得資料行數: 27
高雄市: 12524
取得資料行數: 8
屏東縣: 16128
取得資料行數: 2
臺東縣: 3721
取得資料行數: 7
花蓮縣: 11811
取得資料行數: 2
澎湖縣: 2897
取得資料行數: 5
基隆市: 8048
取得資料行數: 9
新竹市: 14484
取得資料行數: 32
臺中市: 12503
取得資料行數: 4
嘉義市: 11486
取得資料行數: 24
臺南市: 16108
取得資料行數: 36
臺北市: 34238
取得資料行數: 27
高雄市: 12524
取得資料行數: 2
金門縣: 4249


In [78]:
teacherQuery = "SELECT * FROM universities WHERE AcaName LIKE '%師範%'"
teacherList = queryDB("stuData.db", teacherQuery)

bTeacher = 0
gTeacher = 0
for i in teacherList:
    bTeacher = bTeacher + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gTeacher = gTeacher + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("師範 男:", bTeacher,"女:",gTeacher)

techQuery = "SELECT * FROM universities WHERE `Level` LIKE '%四技%'"
techList = queryDB("stuData.db", techQuery)

bTech = 0
gTech = 0
for i in techList:
    bTech = bTech + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gTech = gTech + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("技職 男:", bTech,"女:",gTech)

collegeQuery = "SELECT * FROM universities WHERE `Level` LIKE '%學士%'"
collegeList = queryDB("stuData.db", collegeQuery)

bCollege = 0
gCollege = 0
for i in collegeList:
    bCollege = bCollege + i[0][5] + i[0][7] + i[0][9] + i[0][11] + i[0][13] + i[0][15] +i[0][17]
    gCollege = gCollege + i[0][6] + i[0][8] + i[0][10] + i[0][12] + i[0][14] + i[0][16] +i[0][18]
print("普通 男:", bCollege,"女:",gCollege)

取得資料行數: 12
師範 男: 13032 女: 16299
取得資料行數: 139
技職 男: 233300 女: 213434
取得資料行數: 107
普通 男: 237477 女: 250062


In [65]:
ListMale = []
ListFemale = []

for i in stuList:
    ListMale.extend(map(int,[i[5] , i[7] , i[9] , i[11] , i[13] , i[15] , i[17] , i[19]]))
    ListFemale.extend(map(int,[i[6] , i[8] , i[10] , i[12] , i[14] , i[16] , i[18] , i[20]]))

In [69]:
import numpy
import scipy
mNPList = numpy.array(ListMale)
fNPList = numpy.array(ListFemale)
print("男學生總數:",sum(ListMale),"平均:",numpy.mean(mNPList, axis=0),"標準差:",numpy.std(mNPList,axis=0),"峰度:",scipy.stats.kurtosis(ListMale),"偏態",scipy.stats.skew(ListMale))
print("女學生總數:",sum(ListFemale),"平均:",numpy.mean(fNPList, axis=0),"標準差:",numpy.std(fNPList,axis=0),"峰度:",scipy.stats.kurtosis(ListFemale),"偏態",scipy.stats.skew(ListFemale))

男學生總數: 646866 平均: 95.0155699177 標準差: 253.452331542 峰度: 27.176562477234395 偏態 4.622458027103874
女學生總數: 662575 平均: 97.3230023502 標準差: 262.811386773 峰度: 29.21221456102535 偏態 4.761609223677928
